In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext magic_duckdb



In [ ]:
# Optional: uses keys.ini to load openai_key. You can also just set it directly.

"""Optional: Use a keys.ini file"""
import configparser
import os
import magic_duckdb
from magic_duckdb.extras import sql_ai

keyfile = f"{os.path.dirname(magic_duckdb.__file__)}/keys.ini"

keys = configparser.ConfigParser()
keys.read(keyfile)

openai_key = keys.get("keys", "OPENAI_KEY", fallback=None) # type: ignore

sql_ai.openai_key = openai_key
sql_ai.print_prompts = True


In [ ]:
%%dql
CREATE OR REPLACE TABLE xyz as SELECT * from range(100) t(x);
CREATE OR REPLACE TABLE abc as SELECT * from range(50) y(a);

In [ ]:
%%dql -ai write my sql

x plus a where the total is odd

In [ ]:
%%dql -aichat write my sql

select product of x and a

In [ ]:
# Testing outside the magics
import duckdb
from magic_duckdb.extras import sql_ai

con = duckdb.connect()

con.execute("create table xyz as select * from range(100) t(x)")
con.execute("create table abc as select * from range(100) y(a)")

r = sql_ai.fix_statement(connection=con, command="suggest", statement="a query that combines results from all tables")
print(r)

r = sql_ai.fix_statement(connection=con, command="fix using existing tables and reformat according to SQL Formatting Standard with capital letters and newlines but no trailing slashes", statement="selct cont(*) frm x join a ")
print(r)

In [ ]:

df = con.sql("select * from duckdb_columns").df()

col_desc = []
for t in df["table_name"].unique():
    cols = [f"{v[0]} (type = {v[1]})" for v in df.loc[df["table_name"] == t, ["column_name", "data_type"]].values]
    cols_desc = ",".join(cols)
    
    desc = f"Table {t} columns: {cols_desc}"
    col_desc.append(desc)

col_desc
